In [1]:
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
import selenium.webdriver.support.expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
import time
import sqlite3
import random
from datetime import datetime

from bs4 import BeautifulSoup
import re
#from goose import Goose
#import pymongo
from time import mktime
import os

In [2]:
nrodada = 22

In [3]:
chromedriver = ".\chromedriver\chromedriver"
os.environ["webdriver.chrome.driver"] = chromedriver
chrome_options = Options()
chrome_options.add_argument("--disable-extensions")

browser = webdriver.Chrome(chromedriver,chrome_options=chrome_options)
browser.implicitly_wait(10)
        

In [4]:
browser.get(u'http://cartolafc.globo.com/')
time.sleep(5)

In [5]:
f = open('login.txt')
user = f.readline()
password = f.readline().replace('\r\n','')
f.close()

In [6]:
try:
    entrar = browser.find_element_by_link_text('ENTRAR ›')
    ActionChains(browser).move_to_element(entrar).perform()
    entrar.click()

    time.sleep(1)
    browser.switch_to_frame('login-popin-iframe')
    time.sleep(1)
    browser.find_element_by_xpath('//*[@id="login"]').clear()
    time.sleep(1)
    browser.find_element_by_xpath('//*[@id="login"]').send_keys(user)
    time.sleep(1)
    browser.find_element_by_xpath('//*[@id="password"]').clear()
    time.sleep(1)
    browser.find_element_by_xpath('//*[@id="password"]').send_keys(password)
    time.sleep(2)
    browser.find_element_by_xpath('//*[@id="tpl-content"]/div[2]/div[1]/form/div[6]').click()

    time.sleep(5)
    browser.find_element_by_xpath('//*[@id="botao-entre-cartola"]/span').click()  # Entra no Cartola
except:
    time.sleep(1)
    browser.find_element_by_xpath('//*[@id="login"]').clear()
    time.sleep(1)
    browser.find_element_by_xpath('//*[@id="login"]').send_keys(user)
    time.sleep(1)
    browser.find_element_by_xpath('//*[@id="password"]').clear()
    time.sleep(1)
    browser.find_element_by_xpath('//*[@id="password"]').send_keys(password)
    time.sleep(2)
    browser.find_element_by_xpath('//*[@id="tpl-content"]/div[2]/div[1]/form/div[6]').click()

    time.sleep(5)
    browser.find_element_by_xpath('//*[@id="botao-entre-cartola"]/span').click()  # Entra no Cartola

In [7]:
browser.find_element_by_xpath('/html/body/div/div[7]/main/div/div[3]/div/button').click() # Escalar meu time

In [8]:
opcoes = ['MEIA','ATACANTE', 'ZAGUEIRO','LATERAL','GOLEIRO',u'TÉCNICO']

dict_estados = {'Provavel':'//*[@id="atletas-mercado"]/div[2]/div[2]/div/div/div[2]/div/div[2]/div/span',
                'Duvida': '//*[@id="atletas-mercado"]/div[2]/div[2]/div/div/div[2]/div/div[4]/div/span',
                'Nulo': '//*[@id="atletas-mercado"]/div[2]/div[2]/div/div/div[2]/div/div[6]/div/span'
                }

dict_posicao = {'GOLEIRO' : '//*[@id="atletas-mercado"]/div[2]/div[2]/div/div/div[1]/div/div[2]/div/span',
               'ZAGUEIRO': '//*[@id="atletas-mercado"]/div[2]/div[2]/div/div/div[1]/div/div[3]/div/span',
                'LATERAL': '//*[@id="atletas-mercado"]/div[2]/div[2]/div/div/div[1]/div/div[4]/div/span',
                'MEIA': '//*[@id="atletas-mercado"]/div[2]/div[2]/div/div/div[1]/div/div[5]/div/span',
                'ATACANTE': '//*[@id="atletas-mercado"]/div[2]/div[2]/div/div/div[1]/div/div[6]/div/span',
                u'TÉCNICO': '//*[@id="atletas-mercado"]/div[2]/div[2]/div/div/div[1]/div/div[7]/div/span'
                }

dict_qtde = {'MEIA': 45, 'ATACANTE': 5, 'ZAGUEIRO': 5,'LATERAL': 5,'GOLEIRO' : 5,u'TÉCNICO': 2}
               

In [9]:
browser.find_element_by_xpath('//*[@id="atletas-mercado"]/div[1]/div/div[2]/div/div[2]/span').click()  # botao Filtro
time.sleep(3)
browser.find_element_by_xpath('//*[@id="atletas-mercado"]/div[2]/div[3]/div/div/div[1]').click()  # limpar filtro
time.sleep(2)

In [10]:
def rola_pagina(qtd):
    for i in range(qtd):
        element = browser.find_element_by_xpath('/html/body/div/div[7]')
        ActionChains(browser).move_to_element(element).perform()
        time.sleep(2)

In [11]:
dict_pontuacao = {"atleta.pontos ? atleta.pontos : '-'" : u'Atleta_pontos',
"atleta.media ? atleta.media : '-'": u'Atleta_média',
"atleta.jogos_num ? atleta.jogos_num : '-'": u'Número_Jogos',
"atleta.scout.G": u'Gol',
"atleta.scout.A": 'Assistência', 
"atleta.scout.FT": u'Finalização_na_trave',
"atleta.scout.FD": u'Finalização_defendida',
"atleta.scout.FF": u'Finalização_pra_fora',
"atleta.scout.FS": u'Falta_sofrida',
"atleta.scout.PP": u'Pênalti_perdido',
"atleta.scout.I": u'Impedimento',
"atleta.scout.PE": u'Passe_errado',
"atleta.scout.SG": u'Jogos_sem_sofrer_gols',
"atleta.scout.DP": u'Defesa_de_pênalti',
"atleta.scout.DD": u'Defesa_difícil',
"atleta.scout.RB": u'Roubada_de_bola',
"atleta.scout.GC": u'Gol_contra',
"atleta.scout.CV": u'Cartão_vermelho',
"atleta.scout.CA": u'Cartão_amarelo',
"atleta.scout.GS": u'Gol_sofrido',
"atleta.scout.FC": u'Falta_cometida'}


In [12]:

def scraping_resultado_posicao(soup):
    jogadores = []
    for s in soup.body.findAll(attrs={'ng-repeat': "atleta in timecontroller.atletasInfiniteScroll track by $index"}):

        dados = {}
        dados[u'Nome'] = s.find(attrs={'class': "small-11 medium-8 large-2 columns cartola-atletas__foto-container large-push-2"}).img['alt'] # nome
        dados[u'Time'] = s.find(attrs={'class': "cartola-atletas__time"}).img['title'] # time
        dados[u'Abreviacao'] = s.find(attrs={'class': "show-for-large-up cartola-atletas__time__abreviacao"}).getText() # Abreviação
        dados[u'Apelido'] = s.find(attrs={'class': "columns cartola-atletas__apelido"}).getText() # apelido
        dados[u'Posicao'] =  s.find(attrs={'class': "column cartola-atletas__posicao cartola-atletas__posicao--full"}).getText() # posicao
        try:
            dados[u'seletor'] = s.find(attrs={'class': "cartola-atletas__card-badges"}).svg['seletor']  # seletor
        except:
            dados[u'seletor'] = 'Nulo'

        for s2 in s.findAll(attrs={'class': "cartola-atletas__stats__numeros"}):
            dados[dict_pontuacao[s2['ng-bind']]]= s2.getText()

        dados['preco_cartoleta'] = s.find(attrs={'class': "cartola-atletas__preco-cartoleta"}).text

        try:
            dados['preco_variacao_positiva'] = '+' + s.find(attrs={'class': "cartola-atletas__preco-media cartola-atletas__preco-media--positiva"}).text
        except:
            try:
                dados['preco_variacao_negativa'] = '-' + s.find(attrs={'class': "cartola-atletas__preco-media cartola-atletas__preco-media--negativa"}).text
            except:
                dados['preco_variacao_negativa'] = '-' + s.find(attrs={'class': "cartola-atletas__preco-media"}).text
                pass

        jogadores.append(dados)
    
    return jogadores

In [13]:
resultado_global = []

for i, pos in enumerate(opcoes):
    
    #for e in estados:
    if i == 0:    
        browser.find_element_by_xpath('//*[@id="atletas-mercado"]/div[1]/div/div[2]/div/div[2]/span').click() 
        time.sleep(2)
    else:
        browser.find_element_by_xpath('html/body/div/div[5]/ui-view/div[2]/div[1]/div/div[2]/div[2]').click() 
        time.sleep(2)

    # desmarca a opcao anterior
    if i>0:
        browser.find_element_by_xpath(dict_posicao[opcoes[i-1]]).click()    
    
    # marca a opção desejada
    browser.find_element_by_xpath(dict_posicao[pos]).click()
    time.sleep(1)
    #browser.find_element_by_xpath(dict_estados[e]).click()
    #time.sleep(.5)

    # executa o filtro
    browser.find_element_by_xpath('//*[@id="atletas-mercado"]/div[2]/div[3]/div/div/div[2]').click()
    time.sleep(2)

    rola_pagina(dict_qtde[pos])

    f = browser.page_source
    soup = BeautifulSoup(f, 'html.parser')
    resultado_global.append(scraping_resultado_posicao(soup))
    time.sleep(5)

        

In [14]:
planilha = [r2 for r in resultado_global for r2 in r]
df = pd.DataFrame(planilha)

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 837 entries, 0 to 836
Data columns (total 30 columns):
Atleta_média               837 non-null object
Cartão_amarelo             837 non-null object
Cartão_vermelho            837 non-null object
Defesa_de_pênalti          837 non-null object
Falta_sofrida              837 non-null object
Finalização_na_trave       837 non-null object
Finalização_pra_fora       837 non-null object
Gol                        837 non-null object
Nome                       837 non-null object
Número_Jogos               837 non-null object
Posicao                    837 non-null object
Roubada_de_bola            837 non-null object
preco_cartoleta            837 non-null object
preco_variacao_positiva    130 non-null object
seletor                    837 non-null object
Assistência                837 non-null object
Atleta_pontos              837 non-null object
Jogos_sem_sofrer_gols      837 non-null object
Finalização_defendida      837 non-null object
Ti

In [16]:
df.to_pickle('{}rodada.pkl'.format(str(nrodada)))

In [17]:
### Tabela de Jogos

In [18]:
#browser.find_element_by_xpath('//*[@id="atletas-mercado"]/div[1]/div/div[2]/div/div[1]/span').click()
browser.find_element_by_xpath('/html/body/div[1]/div[5]/ui-view/div[2]/div[1]/div/div[2]/div[1]/span').click()

In [19]:
f = browser.page_source
soup = BeautifulSoup(f, 'html.parser')  

In [20]:
r = []


for s in soup.findAll(attrs={'class': 'jogos-rodada__partidas'}):
#for s in soup.findAll(attrs={'class': 'cartola-popin-body-content cartola-card'}):
    
    for s2 in s.find_all('li'):
        
        dados = {}
        dados['data'] =  s2.find(attrs={'class': "jogos-rodada__partida__data"}).text
        dados['local'] =  s2.find(attrs={'class': "jogos-rodada__partida__local"}).text
        dados['hora'] = s2.find(attrs={'class': "jogos-rodada__partida__hora"}).text
        dados['Mandante_posicao'] = s2.find(attrs={'class': "jogos-rodada__partida__mandante__posicao column small-2"}).text
        dados['Mandante'] = s2.find(attrs={'class': "column small-3"}).img['alt']
        dados['Visitante_posicao'] = s2.find(attrs={'class': "jogos-rodada__partida__visitante__posicao column small-2"}).text
        dados['Visitante'] = s2.findAll(attrs={'class': "column small-3"})[1].img['alt']

        r.append(dados)
        
        
    

In [21]:
proximo_jogos = r

In [22]:
df2  = pd.DataFrame(proximo_jogos)

In [23]:
df2.to_pickle('{}rodada_jogos.pkl'.format(str(nrodada+1)))

In [24]:
df2

,Mandante,Mandante_posicao,Visitante,Visitante_posicao,data,hora,local
0,Atlético-MG,11º,Palmeiras,4º,SAB 09/09/2017,16:00,Independência
1,São Paulo,19º,Ponte Preta,13º,SAB 09/09/2017,19:00,Morumbi
2,Vasco,12º,Grêmio,2º,DOM 10/09/2017,00:00,São Januário
3,Atlético-PR,8º,Coritiba,15º,DOM 10/09/2017,11:00,Arena da Baixada
4,Santos,3º,Corinthians,1º,DOM 10/09/2017,16:00,Vila Belmiro
5,Sport,10º,Avaí,18º,DOM 10/09/2017,16:00,Ilha do Retiro
6,Vitória,16º,Fluminense,9º,DOM 10/09/2017,16:00,Barradão
7,Botafogo,7º,Flamengo,5º,DOM 10/09/2017,19:00,Engenhão
8,Chapecoense,17º,Cruzeiro,6º,DOM 10/09/2017,19:00,Arena Condá
9,Atlético-GO,20º,Bahia,14º,SEG 11/09/2017,20:00,Olímpico (GO)


# Captura Resultados

In [25]:
browser.get('http://globoesporte.globo.com/futebol/brasileirao-serie-a/')
time.sleep(5)

In [26]:
f = browser.page_source
soup = BeautifulSoup(f, 'html.parser')

def get_back(soup):
    while soup.body.find(attrs={'class':'tabela-navegacao-seletor'}).text != u'01\xaa RODADA':
        browser.find_element_by_xpath('//*[@id="container-para-tabela-simulador-ou-navegacao-js"]/article/aside/nav/span[1]').click()
        f = browser.page_source
        soup = BeautifulSoup(f, 'html.parser')
        time.sleep(1)

get_back(soup)

In [27]:
jogos = []
for i in range(nrodada): # ate a 6º rodada
    
    f = browser.page_source
    soup = BeautifulSoup(f, 'html.parser')
    
    rodada = soup.body.find(attrs={'class':'tabela-navegacao-seletor'}).text
    for t in soup.body.findAll(attrs={'class':'placar-jogo'}):
        jogo = {}
        jogo['rodada'] = rodada
        jogo['jogo']= t.find(attrs={'itemprop':'name'})['content']  #Jogo
        jogo['Mandante'] =  t.find(attrs={'class':'placar-jogo-equipes-escudo-mandante'})['title']  #Mandante
        jogo['Mandante_Score'] =  t.find(attrs={'class':'placar-jogo-equipes-placar-mandante'}).text # Mandante Score
        jogo['Visitante'] =  t.find(attrs={'class':'placar-jogo-equipes-escudo-visitante'})['title']  # Visitante
        jogo['Visitante_Score'] = t.find(attrs={'class':'placar-jogo-equipes-placar-visitante'}).text   #Visitante Score
        jogos.append(jogo)

    
    browser.find_element_by_xpath('//*[@id="container-para-tabela-simulador-ou-navegacao-js"]/article/aside/nav/span[3]').click()
    time.sleep(5)

In [28]:
pd.DataFrame(jogos).to_excel(r'.\Excel\resultados.xlsx')
pd.DataFrame(jogos).to_pickle('resultados.pkl')